# Lody w Warszawie - where to open an ice cream shop in Warsaw

## Table of contents
* [Introduction / Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction / Business Problem <a name="introduction"></a>

As average temperatures rise on an yearly basis in Warsaw, selling ice cream is the type of business that can bring significant profits, especially in the heated summer months. Natural ice cream (or "lody naturalne", as locals call it) is becoming more and more popular and **entreprenuers** with very well crafted recipes and marketing strategies are asking "where should I open my shop?", as the **location of the shop could exponentially impact the returns**.

Through this project, I intend to help such entrepreneurs make the right decision when it comes to choosing the location to open their shop using **geospatial data visualisation** such as **choropleth maps** and **KDE (Kernel Density Estimation) heatmaps**. 

## Data<a name="data"></a>

Based on the definition of our problem, factors that will influence our decision are:
* population density in a given district of Warsaw;
* number of existing ice cream shops in the district;
* distance of the district the from city center.

The data used to solve this problem is:
* geospatial data for **each of the 18 districts in Warsaw** and **the most recent population statistics**; 
* geospatial data about **venues selling ice cream in Warsaw** collected from FourSquare. 

Our dataset contains location data (*lat*, *lng*), where *lat* stands for *latitude* and *lng* for *longitude*, population, area and population per km2 for each of the 18 districts of Warsaw. Furthermore, we will calculate the centroid of each district in order to compute the distance from the city center of each district.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import geopandas as gpd # library to create geopanda dataframes 
import pygeoj # library to import data from geojson file
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library
import folium.plugins as plugins

import mpu #library to calculate distance between 2 coordinates

import geopandas.tools
from shapely.geometry import Point

## Geospatial data and population statistics for Warsaw districts

First we will import our geojson file with coordinates of each Warsaw district.

In [2]:
# Special thanks to Andrzej Kostanski (github.com/andilabs)
geojson_data = pygeoj.load(filepath="warszawa-dzielnice2.geojson")

Then we will create a geodataframe and add the centroids.

In [3]:
gdf_districts = gpd.read_file("warszawa-dzielnice2.geojson",encoding= 'unicode_escape')
gdf_districts = gdf_districts.to_crs(epsg=4326) # format spatial reference system
gdf_districts['centroid_lon'] = gdf_districts['geometry'].centroid.x
gdf_districts['centroid_lat'] = gdf_districts['geometry'].centroid.y

<ipython-input-3-8a6b68d62ae3>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_districts['centroid_lon'] = gdf_districts['geometry'].centroid.x
<ipython-input-3-8a6b68d62ae3>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_districts['centroid_lat'] = gdf_districts['geometry'].centroid.y


Next, we will import the most recent (2019) population density data for the 18 districts of Warsaw and merge it with our geodataframe. Data has been downloaded from GUS (Główny Urząd Statystyczny).

In [4]:
# import 2019 population density data for Warsaw districts
population = pd.read_csv('population per km2.csv',encoding= 'unicode_escape')
population[["Population"]] = population[["Population"]].apply(pd.to_numeric)
population[["Area"]] = population[["Area"]].apply(pd.to_numeric)
population[["Pop_per_km2"]] = population[["Pop_per_km2"]].apply(pd.to_numeric)

# merge this data to geodataframe
gdf_districts_population = gdf_districts.merge(population, left_on='name', right_on='District')

Calculate the distance between each district and the Warsaw city center and add it to the dataframe.

In [5]:
gdf_districts_population['distance'] = gdf_districts_population.apply(lambda x: mpu.haversine_distance((x['centroid_lat'], x['centroid_lon']), (52.237049, 21.017532)), axis=1)

## Foursquare

In [6]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20200620' # Foursquare API version

In [7]:
LIMIT = 300 # limit number of venues returned by Foursquare API

radius = 10000 # define radius (10000 meters)

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    52.237049, 21.017532, # coordinates of Warsaw city centre
    radius, 
    LIMIT,
    '4bf58dd8d48988d1c9941735' # Ice cream shop venue code
    )

In [8]:
results = requests.get(url).json() #json output from Foursquare

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# create dataframe
df_venues = pd.DataFrame(nearby_venues, columns=["name", "categories", "lat", "lng"])
df_venues.head()

<ipython-input-10-cb4b831388b3>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,N'Ice Cream Factory,Ice Cream Shop,52.232856,21.017677
1,Lodziarnia Ulica Baśniowa,Ice Cream Shop,52.262135,20.980712
2,Jednorożec Lody tradycyjne,Ice Cream Shop,52.205944,21.013504
3,Lodziarnia Malinova,Ice Cream Shop,52.203823,21.009882
4,Na Końcu Tęczy,Ice Cream Shop,52.219837,21.019050


How many venues that sell ice cream were returned by Foursquare?

In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

96 venues were returned by Foursquare.


In [12]:
unique_categories = df_venues['categories'].unique()
print('The following venue categories sell ice cream in Warsaw:')
for i in range(0, len(unique_categories)):    
    print(unique_categories[i])

The following venue categories sell ice cream in Warsaw:
Ice Cream Shop
Dessert Shop
Chocolate Shop
Café
French Restaurant
Bakery
Asian Restaurant
Coffee Shop
Restaurant
Scandinavian Restaurant
Vegetarian / Vegan Restaurant
Gastropub
Pizza Place
Juice Bar
Fast Food Restaurant


Let's see how many venues of each category sell ice cream.

In [13]:
print(df_venues.groupby(['categories'], sort=False).size().reset_index(name='Count'))

                       categories  Count
0                  Ice Cream Shop     65
1                    Dessert Shop      6
2                  Chocolate Shop      2
3                            Café      9
4               French Restaurant      1
5                          Bakery      2
6                Asian Restaurant      1
7                     Coffee Shop      1
8                      Restaurant      3
9         Scandinavian Restaurant      1
10  Vegetarian / Vegan Restaurant      1
11                      Gastropub      1
12                    Pizza Place      1
13                      Juice Bar      1
14           Fast Food Restaurant      1


As expected, most venues in our dataset are ice cream shops, with a few venues such as Restaurants or Bakeries also selling a variety of ice cream sortiments. 

Next, we will assign the district for each shop based on its coordinates and then count the shops in each district.

In [14]:
# Create the geometry column from the coordinates
df_venues["geometry"] = df_venues.apply(lambda row: Point(row["lng"], row["lat"]), axis=1)

# Convert to a GeoDataFrame
gdf_venues = gpd.GeoDataFrame(df_venues, geometry="geometry")

# Declare the coordinate system for the places GeoDataFrame
# GeoPandas doesn't do any transformations automatically when performing
# the spatial join. The layers are already in the same CRS (WGS84) so no
# transformation is needed.
gdf_venues.crs = {"init": "epsg:4326"}

# Drop all columns except the name and polygon geometry
venue_district = gdf_venues[["name", "geometry"]]

# Perform the spatial join
result = geopandas.tools.sjoin(venue_district, gdf_districts, how="left")

# Create new dataframe and count shops by district 
gdf_count_shops = result.groupby('name_right').agg({'name_right': 'count'})
gdf_count_shops = gdf_count_shops.rename(columns={"name_right": "Number of shops"})
gdf_count_shops
# Add the number of ice cream shop per district to our initial geodataframe
gdf_merged = gdf_districts_population.merge(gdf_count_shops, left_on='name', right_on='name_right', how="left")
gdf_merged = gdf_merged.fillna(0)
gdf_merged2 = gdf_merged[['District','Pop_per_km2','distance','Number of shops']]
gdf_merged2.sort_values("Number of shops", ascending = False)

C:\Users\Acer1\anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<ipython-input-14-5e41bc741a7d>:17: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = geopandas.tools.sjoin(venue_district, gdf_districts, how="left")


,District,Pop_per_km2,distance,Number of shops
9,Srodmiescie,7336,0.483060,36.0
6,Mokotow,6166,5.195829,16.0
7,Praga Poludnie,8071,4.575237,12.0
15,Wilanow,1148,11.188729,6.0
0,Ochota,8504,4.176623,4.0
17,Zoliborz,6211,4.275623,4.0
5,Bielany,4074,8.809471,3.0
12,Wola,7365,4.131700,3.0
2,Bemowo,5025,7.798620,3.0
8,Praga Polnoc,5569,3.160067,2.0


As we can see in the table above, Srodmiescie, Mokotow and Praga Poludnie are the districts with the most number of shops that sell ice cream. 

## Methodology<a name="methodology"></a>

In this project we will direct our efforts on detecting the districts of Warsaw that can be suitable for opening a new ice cream shop, taking into consideration factors such as population density, number of existing ice cream shops in the district or distance from the city center.

As a first step in the analysis, we will create Folium maps to analyze our geospatial data. Specifically, we will create **choropleth maps** and **KDE (Kernel Density Estimation) heatmaps** to assess the demographical aspects of Warsaw's districts and determine the area of influence of the existing ice cream shops. 

## Analysis <a name="analysis"></a>

Having created the geodataframe, we can now use the Folium library to create a map of Warsaw with the districts and their population statistics.

In [15]:
m = folium.Map(location=[np.median(gdf_districts_population['centroid_lat'].tolist()), 
                         np.median(gdf_districts_population['centroid_lon'].tolist())], tiles='CartoDB positron', zoom_start=11)
folium.Choropleth(
    geo_data=geojson_data,
    data=population,
    columns=['District', 'Pop_per_km2'],
    key_on='feature.properties.name',
    fill_color='GnBu',
    bins=[900, 3000, 5000, 7000, 9000],
    fill_opacity=0.8,
    line_opacity=0.8,
    legend_name='Population density',
    highlight=True
).add_to(m)


# add markers with basic information
fg = folium.FeatureGroup(name='District Info')
for lat, lon, pop, area, pop_km2, name in zip(gdf_districts_population['centroid_lat'].tolist(), gdf_districts_population['centroid_lon'].tolist(), gdf_districts_population['Population'].tolist(), gdf_districts_population['Area'].tolist(), gdf_districts_population['Pop_per_km2'].tolist(), gdf_districts_population['name'].tolist()):
    html = f"""
    <h2>{name}<\h2>
    <h6>Population: {int(round(pop,0))}<\h6>
    <h6>Area: {area} km2<\h6>
    <h6>Population density: {int(round(pop_km2,0))} people/km2 <\h6>
    """
    fg.add_child(folium.Marker(location=[lat, lon], popup=html))

m.add_child(fg)

# enable layers to be turned in or out
folium.LayerControl().add_to(m)

m

After amalyzing the map above, we can observe that the most densely populated districts of Warsaw are: Srodmiescie, Wola, Ochota and Praga Poludnie, whereas the least densely populated districts are at the outskirts, such as Wilanow or Ursus.

Next, let's create a heatmap to have a better view of the regarding the spread of ice cream shops in Warsaw.

In [16]:
from folium.plugins import HeatMap

# create map of Warsaw using latitude and longitude values
hmap_warsaw = folium.Map(location=[52.237049, 21.017532], zoom_start=12, tiles = 'CartoDB positron')

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=3,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.2,
            parse_html=False).add_to(hmap_warsaw)
        
# generate choropleth map 
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    fill_opacity=0.0,
    line_opacity=0.9,
    highlight=False
).add_to(hmap_warsaw)

# add labels indicating the name of the district
style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], style=style_function, labels=False))

# create a layer control
folium.LayerControl().add_to(m)


add_markers(df_venues)
hm_data = df_venues[["lat", "lng"]].to_numpy().tolist()
hmap_warsaw.add_child(plugins.HeatMap(hm_data))
hmap_warsaw.add_child(folium.ClickForMarker(popup='Potential Location'))
hmap_warsaw

The map above gives us a better overview of the number of shops in each district and we can clearly see how the majority of ice cream shops concentrate in the districts of Srodmiescie, Zoliborz, Mokotow, Ochota and Praga Poludnie.

## Results and Discussion <a name="results"></a>

Taking into account all the information gathered so far, Wola seems the best district to open an ice cream shop at the moment as the population density is very high compared to other districts and the number of ice cream shops is significantly lower than in districts such as Mokotow or Srodmiescie.

In [17]:
from folium.plugins import HeatMap

# create map of Warsaw using latitude and longitude values
hmap_warsaw = folium.Map(location=[52.237049, 21.017532], zoom_start=12, tiles = 'CartoDB positron')

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=3,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.2,
            parse_html=False).add_to(hmap_warsaw)
        
# generate choropleth map 
choropleth = folium.Choropleth(
    geo_data=geojson_data,
    fill_opacity=0.0,
    line_opacity=0.9,
    highlight=False
).add_to(hmap_warsaw)

# add labels indicating the name of the district
style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], style=style_function, labels=False))

# create a layer control
folium.LayerControl().add_to(m)


add_markers(df_venues)
hm_data = df_venues[["lat", "lng"]].to_numpy().tolist()
hmap_warsaw.add_child(plugins.HeatMap(hm_data))
hmap_warsaw.add_child(folium.ClickForMarker(popup='Potential Location'))
hmap_warsaw

On the map above, we have marked possible locations in the district of Wola that we consider great locations for opening an ice cream shop. All 3 are at the are at street intersections and are either closer to the city center or in a residential area, with no other ice cream shops nearby.

For further analysis, we suggest to take into account other variables while creating the choropleth, such as average income level in the district. 

## Conclusion <a name="conclusion"></a>

The purpose of this project was to identify posiible locations to open a new ice cream shop in the city of Warsaw. By gathering enough geospatial data from Foursquare and GUS, we have managed to create choropleth and KDE (Kernel Density Estimation) heatmaps that have facilitated our analysis. Lastly, we have made specific recomendations by marking posiible locations in the district of Wola.

The final decision regarding the optimal ice cream shop location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.